In [150]:
import pandas as pd
import geopandas as gpd
from pyproj import CRS
from shapely.geometry import Polygon
from shapely.geometry import Point
import rasterio as rio
from rasterio import features
from rasterio.windows import Window
import matplotlib.pyplot as plt
import numpy as np
import pprint as pp
import itertools as it
from tqdm import tqdm
import time


In [ ]:
from rasterio.plot import show
from rasterio.warp import transform

xs = np.array([3327493.433, 16177493.43])
ys = np.array([7389201.61, -580798.392])

lossyear = 'data/Hansen_GFC-2022-v1.10_lossyear_50N_000E.tif'
datamask = 'data/Hansen_GFC-2022-v1.10_datamask_50N_000E.tif'
foo = 'data/TCL_DD_2022_20230407.tif'

src = rio.open(foo)
band = src.read(1)



In [ ]:
mask = band != 12
shapes = features.shapes(band, mask=mask, transform=src.transform)
pp.pprint(next(shapes)) # first element
    

({'coordinates': [[(1.4645000000000001, 50.0),
                   (1.4645000000000001, 49.99975),
                   (1.46525, 49.99975),
                   (1.46525, 49.9995),
                   (1.4655, 49.9995),
                   (1.4655, 50.0),
                   (1.4645000000000001, 50.0)]],
  'type': 'Polygon'},
 5.0)

In [ ]:
# rasterio.features.dataset_features # works on raster value rather than mask...

with rio.open('data/TCL_DD_2022_20230407.tif') as src:
    # TODO: check src.count for number of bands...
    print(src.meta)
    band = src.read(1)
    mask = band == 1
    # Object holding a feature collection that implements the __geo_interface__
    # TODO: result should be in EPSG 4326 i.e. GPS
    results = (
        {'properties': {'deforestation': v}, 'geometry': s}
        for i, (s, v) in enumerate(
            features.shapes(band, mask=mask)
            )
        )
    geoms=list(results)
    gdf = gpd.GeoDataFrame.from_features(geoms)
    
gdf.head()


geometry	deforestation
0	POLYGON ((2870.000 104.000, 2870.000 105.000, ...	4.0
1	POLYGON ((2777.000 110.000, 2777.000 111.000, ...	4.0
2	POLYGON ((2778.000 111.000, 2778.000 112.000, ...	4.0
3	POLYGON ((2730.000 113.000, 2730.000 114.000, ...	4.0
4	POLYGON ((2737.000 113.000, 2737.000 114.000, ...	4.0

In [142]:
# rasterio.features.dataset_features # works on raster value rather than mask...

with rio.open('data/Hansen_GFC-2022-v1.10_lossyear_20S_060W.tif') as src:
    # TODO: check src.count for number of bands...
    print(src.meta)
    print(f'crs: {src.crs}')
    print(f'is_epsg_code: {src.crs.is_epsg_code}')
    if src.crs.is_epsg_code:
        print(f'epsg: {src.crs.to_epsg}')
    print(f'is_geographic: {src.crs.is_geographic}')
    print(f'is_projected: {src.crs.is_projected}')
    print(f'linear_units: {src.crs.linear_units}')
    if src.crs.is_projected:
        print(f'linear_units_factor: {src.crs.linear_units_factor}')
    band = src.read(1, window=Window(2100, 2000, 500, 500))
    #band = src.read(1)
    print(band.shape)
    #rdf = gpd.GeoDataFrame()
    mask = band != 0
    # Object holding a feature collection that implements the __geo_interface__
    # TODO: result should be in EPSG 4326 i.e. GPS
    results = (
        {'properties': {'lossyear': v}, 'geometry': s}
        for i, (s, v) in enumerate(
            # connectivity, 4 on edges, 8 on edges and corners...
            features.shapes(band, mask=mask, connectivity=8, transform=src.transform)
            )
        )
    geoms=list(results)
    rdf = gpd.GeoDataFrame.from_features(geoms, crs=src.crs)
    # TODO: tge crs may not 
    #rdf = gpd.GeoDataFrame( pd.concat( [rdf, gdf], ignore_index=True), crs=gdf.crs)

print(f'Chosen Window results in GeoDataFrame of .shape: {rdf.shape}')
rdf.head()


{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 40000, 'height': 40000, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.00025, 0.0, -60.0,
       0.0, -0.00025, -20.0)}
crs: EPSG:4326
is_epsg_code: True
epsg: <bound method CRS.to_epsg of CRS.from_epsg(4326)>
is_geographic: True
is_projected: False
linear_units: unknown
(500, 500)
Chosen Window results in GeoDataFrame of .shape: (1499, 2)


,geometry,lossyear
0,"POLYGON ((-60.00000 -20.00000, -60.00000 -20.0...",22.0
1,"POLYGON ((-59.99200 -20.00000, -59.99200 -20.0...",22.0
2,"POLYGON ((-59.97875 -20.00000, -59.97800 -20.0...",13.0
3,"POLYGON ((-59.97725 -20.00000, -59.97725 -20.0...",22.0
4,"POLYGON ((-59.96225 -20.00000, -59.96225 -20.0...",22.0


In [ ]:
rdf.plot(column='lossyear', legend=True)

In [143]:
# sjoin does not modify the geometry...
intersects = rdf.sjoin(rdf, how="left", predicate="intersects")
intersects.shape, intersects.index.value_counts()

((4169, 4),
 809     148
 1445     80
 1011     63
 1165     54
 357      48
        ... 
 913       1
 916       1
 922       1
 923       1
 1498      1
 Name: count, Length: 1499, dtype: int64)

In [144]:
intersects.reset_index(inplace=True)
intersects.rename(columns={'index': 'index_left'}, inplace=True)
intersects.head()


,index_left,geometry,lossyear_left,index_right,lossyear_right
0,0,"POLYGON ((-60.00000 -20.00000, -60.00000 -20.0...",22.0,0,22.0
1,1,"POLYGON ((-59.99200 -20.00000, -59.99200 -20.0...",22.0,21,10.0
2,1,"POLYGON ((-59.99200 -20.00000, -59.99200 -20.0...",22.0,22,10.0
3,1,"POLYGON ((-59.99200 -20.00000, -59.99200 -20.0...",22.0,23,10.0
4,1,"POLYGON ((-59.99200 -20.00000, -59.99200 -20.0...",22.0,18,4.0


In [145]:
# Nota bene: Robert Norris - this removes the self-intersection aggregates index_right, but
# also aggregates all other values into lists... it also spends far too much time in unary_union
# on geometry, the result of which we do not need.
#temp = intersects.dissolve("index_left", aggfunc=lambda x: x.tolist(),)

# Group by 'index_left', truncate intersects, then aggregate on 'index_right' only...
groups = intersects.groupby('index_left')
temp = intersects[intersects['index_left'] == intersects['index_right']].set_index('index_left')
temp['indices'] = groups['index_right'].aggregate(lambda x: x.tolist())

temp.index.name = None
temp['lossyear'] = temp['lossyear_left'].astype("int") + 2000
temp.drop(['index_right', 'lossyear_left', 'lossyear_right'], axis=1, inplace=True)

temp.head()


,geometry,indices,lossyear
0,"POLYGON ((-60.00000 -20.00000, -60.00000 -20.0...",[0],2022
1,"POLYGON ((-59.99200 -20.00000, -59.99200 -20.0...","[21, 22, 23, 18, 1, 11, 32, 2]",2022
2,"POLYGON ((-59.97875 -20.00000, -59.97800 -20.0...","[1, 2]",2013
3,"POLYGON ((-59.97725 -20.00000, -59.97725 -20.0...","[3, 32]",2022
4,"POLYGON ((-59.96225 -20.00000, -59.96225 -20.0...","[12, 4]",2022


In [43]:
type(temp.indices[0][0])

int

In [146]:

group_id = 'group'

# See https://stackoverflow.com/questions/73566774/group-by-and-combine-intersecting-overlapping-geometries-in-geopandas
# This is not quite right... 5 should be part of the same group as 2, but since it was
# not previously encountered, a new group_id is taken that...
# we would need to take any of 'indices' that have previously been encountered...

'''
                 indices group
0                    [0]     0
1                    [1]     1
2                 [2, 3]     2
3           [2, 3, 4, 6]     2
4        [3, 4, 6, 7, 9]     2
5                 [5, 6]  None
6  [3, 4, 5, 6, 7, 8, 9]     2
                 indices group
0                    [0]     0
1                    [1]     1
2                 [2, 3]     2
3           [2, 3, 4, 6]     2
4        [3, 4, 6, 7, 9]     2
5                 [5, 6]     5
6  [3, 4, 5, 6, 7, 8, 9]     2
                 indices group
0                    [0]     0
1                    [1]     1
2                 [2, 3]     2
3           [2, 3, 4, 6]     2
4        [3, 4, 6, 7, 9]     2
5                 [5, 6]     5
6  [3, 4, 5, 6, 7, 8, 9]     5                 
'''

index_generator = range(len(temp))
start = time.time()
counter = it.count()

indices = temp['indices'].to_numpy()
groups = pd.Series([None] * len(temp))

for i, array in tqdm(zip(counter, indices), total=len(temp)):
    first_valid_index = groups.loc[array].first_valid_index()
    id = i if first_valid_index == None else groups.loc[first_valid_index]
    groups.loc[array] = id
end = time.time()
print(f'Loop over {len(temp)} took {end-start}s')

temp[group_id] = groups.copy()
temp.drop('indices', axis=1, inplace=True)

# want to dissolve based on lossyear to generate any MULTIPOLYGON from disjoint geometry from same lossyear...
temp2 = temp.dissolve(
    [group_id, 'lossyear']
)
#temp.reset_index(inplace=True)
print(f'intersects.dissolve on group_id, lossyear in GeoDataFrame of .shape: {temp2.shape}')

temp2.head(30)
#temp['indices']




  0%|          | 0/1499 [00:00<?, ?it/s]

100%|██████████| 1499/1499 [00:03<00:00, 435.59it/s]


Loop over 1499 took 3.4490370750427246s
intersects.dissolve on group_id, lossyear in GeoDataFrame of .shape: (596, 1)


geometry
group lossyear                                                   
0     2022      POLYGON ((-60.00000 -20.00000, -60.00000 -20.0...
1     2002      POLYGON ((-59.98575 -20.00025, -59.98575 -20.0...
      2004      POLYGON ((-59.98400 -20.00050, -59.98400 -20.0...
      2010      MULTIPOLYGON (((-59.98900 -20.00075, -59.98900...
      2013      POLYGON ((-59.97875 -20.00000, -59.97800 -20.0...
4     2001      POLYGON ((-59.98900 -20.00875, -59.98900 -20.0...
      2002      MULTIPOLYGON (((-59.98025 -20.02875, -59.98025...
      2003      POLYGON ((-59.98175 -20.01050, -59.98175 -20.0...
      2004      MULTIPOLYGON (((-59.98875 -20.02550, -59.98875...
      2005      MULTIPOLYGON (((-59.95350 -20.01825, -59.95350...
      2006      MULTIPOLYGON (((-59.96375 -20.04100, -59.96400...
      2007      MULTIPOLYGON (((-59.97225 -20.02075, -59.97225...
      2008      POLYGON ((-59.97925 -20.00600, -59.97925 -20.0...
      2010      MULTIPOLYGON (((-59.97925 -20.05200, -59.97900...
      2011      MULTIPOLYGON (((-59.98175 -20.03150, -59.98150...
      2012      MULTIPOLYGON (((-59.97925 -20.04550, -59.97925...
      2013      MULTIPOLYGON (((-59.97875 -20.04575, -59.97900...
      2014      MULTIPOLYGON (((-59.99025 -20.05275, -59.98950...
      2018      MULTIPOLYGON (((-59.97950 -20.04525, -59.97950...
      2019      MULTIPOLYGON (((-59.98250 -20.03775, -59.98225...
      2020      MULTIPOLYGON (((-59.98050 -20.02275, -59.98075...
      2021      MULTIPOLYGON (((-59.98900 -20.01625, -59.98825...
      2022      MULTIPOLYGON (((-59.98950 -20.02250, -59.98950...
5     2012      MULTIPOLYGON (((-59.93800 -20.00000, -59.93800...
      2014      MULTIPOLYGON (((-59.93625 -20.00025, -59.93625...
      2022      MULTIPOLYGON (((-59.93400 -20.00000, -59.93400...
8     2012      POLYGON ((-59.93125 -20.00000, -59.93125 -20.0...
9     2012      POLYGON ((-59.88350 -20.00000, -59.88350 -20.0...
10    2010      POLYGON ((-59.88275 -20.00025, -59.88275 -20.0...
      2012      POLYGON ((-59.88275 -20.00000, -59.88275 -20.0...

In [65]:
temp2.head()

geometry
group lossyear                                                   
0     2022      POLYGON ((0.000 0.000, 0.000 3.000, 4.000 3.00...
1     2001      POLYGON ((44.000 35.000, 44.000 36.000, 45.000...
      2002      MULTIPOLYGON (((59.000 1.000, 57.000 1.000, 57...
      2003      MULTIPOLYGON (((73.000 43.000, 74.000 43.000, ...
      2004      MULTIPOLYGON (((64.000 2.000, 64.000 3.000, 65...

In [147]:
group_ids = temp2.index.get_level_values(0).unique()
lossyears = range(2001, 2023)

#temp3 = temp2.set_index([group_id, 'lossyear'])
index = pd.MultiIndex.from_tuples(tuples=it.product(group_ids, lossyears), names=(group_id, 'lossyear'))
temp3 = temp2.reindex(index)

temp3.head(23)

geometry
group lossyear                                                   
0     2001                                                   None
      2002                                                   None
      2003                                                   None
      2004                                                   None
      2005                                                   None
      2006                                                   None
      2007                                                   None
      2008                                                   None
      2009                                                   None
      2010                                                   None
      2011                                                   None
      2012                                                   None
      2013                                                   None
      2014                                                   None
      2015                                                   None
      2016                                                   None
      2017                                                   None
      2018                                                   None
      2019                                                   None
      2020                                                   None
      2021                                                   None
      2022      POLYGON ((-60.00000 -20.00000, -60.00000 -20.0...
1     2001                                                   None

In [ ]:
# Must be aware of chained indexing as it may call __get_item__ before __set_item__ which will fail on Nan/None etc.
# https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

#temp.index.names, temp.index.levels, temp.index.codes
#temp.isna() # shows only rows where at least one column value is set...
#temp.fillna(0)
#temp.loc[(1, 2018), 'geometry'] = None
#temp.loc[(1, 2017), 'index_right'] = 666
#temp.loc[:,].isnull().sum()
#temp.loc[temp.loc[:,].isna(), 'geometry']
#temp.geometry.fillna()
#temp.head()
#temp.loc[1,2019].geometry = None #shapely.geometry.Polygon([])


In [148]:
# geodetic coordinates (e.g. 4826) to meters (e.g. 3857) and vice-versa

temp3.loc[temp3['geometry'].isna(), 'geometry'] = Polygon([])
proj_3857 = temp3.to_crs(epsg=3347) # lambert projection
print(proj_3857.crs)
temp3['area'] = proj_3857.geometry.area
#temp3['cum_area'] = proj_3857.groupby(group_id).area.cumsum()

#for i in tqdm(group_ids.to_numpy()):
#    temp3.loc[i, 'cum_geometry'] = list(it.accumulate(temp3.loc[i, 'geometry'], func=lambda x,y: x.union(y)))

temp3.reset_index(inplace=True)

temp3.drop(temp3[temp3['area'] == 0].index, inplace=True)

print(f'...and a final GeoDataFrame of .shape: {temp3.shape}')

#temp3.to_file('data/geoply-sample.shp')
temp3.head(30)

EPSG:3347
...and a final GeoDataFrame of .shape: (596, 4)


,group,lossyear,geometry,area
21,0,2022,"POLYGON ((-60.00000 -20.00000, -60.00000 -20.0...",2.422444e+05
23,1,2002,"POLYGON ((-59.98575 -20.00025, -59.98575 -20.0...",7.814028e+03
25,1,2004,"POLYGON ((-59.98400 -20.00050, -59.98400 -20.0...",3.907053e+03
31,1,2010,"MULTIPOLYGON (((-59.98900 -20.00075, -59.98900...",1.172127e+04
34,1,2013,"POLYGON ((-59.97875 -20.00000, -59.97800 -20.0...",4.688438e+04
44,4,2001,"POLYGON ((-59.98900 -20.00875, -59.98900 -20.0...",3.908332e+03
45,4,2002,"MULTIPOLYGON (((-59.98025 -20.02875, -59.98025...",1.290358e+05
46,4,2003,"POLYGON ((-59.98175 -20.01050, -59.98175 -20.0...",3.908604e+03
47,4,2004,"MULTIPOLYGON (((-59.98875 -20.02550, -59.98875...",1.798049e+05
48,4,2005,"MULTIPOLYGON (((-59.95350 -20.01825, -59.95350...",7.818139e+03


In [134]:
proj_3857.crs.axis_info

[Axis(name=Easting, abbrev=E, direction=east, unit_auth_code=EPSG, unit_code=9001, unit_name=metre),
 Axis(name=Northing, abbrev=N, direction=north, unit_auth_code=EPSG, unit_code=9001, unit_name=metre)]

In [149]:
print(temp3.dtypes)
#type(temp3['indices'][0])
#type(temp3['cum_geometry'].dtypes)
temp3.to_file('data/geoply-sample.gpkg', driver='GPKG')

group          int64
lossyear       int64
geometry    geometry
area         float64
dtype: object


In [175]:
#pt = Point(13.404954, 52.520008)
pt = Point(-21, -60)
mask = temp3.geometry.contains(pt)
first_valid_index = None if not mask.any() else temp3[mask].index[0]

print(first_valid_index)

None
